In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# df = pd.read_parquet("metrics_per_sample.parquet")

df = pd.read_parquet(
    "/lustre/scwpod02/client/kyutai/vaclav/data/tts_longform_debug/2/metrics.parquet"
)
df["gender"] = df["tags"].apply(lambda x: "f" if "gender=f" in x else "m")
df = df.drop(
    columns=["tags", "file", "nn", "n_q0.25", "n_q0.5", "n_q0.75", "n_q1.0", "language"]
)
# Reset index but also change its order to match the file paths
df = df.reset_index().set_index(["dataset", "method", "sample_id"]).reset_index()

In [ ]:
df

In [ ]:
quantiles = [0.25, 0.5, 0.75, 1.0]
wer_cols = [f"w_q{q}" for q in quantiles]
sim_cols = [f"s_q{q}" for q in quantiles]

# Build a list of DataFrames for each quantile
dfs = []
for q, w_col, s_col in zip(quantiles, wer_cols, sim_cols):
    temp = df.copy()
    temp["quantile"] = q
    temp["wer"] = temp[w_col]
    temp["sim"] = temp[s_col]
    dfs.append(temp)

# Concatenate and select relevant columns
df_quantile = pd.concat(dfs)
df_quantile = df_quantile.drop(columns=wer_cols + sim_cols)
df_quantile

In [ ]:
df_quantile

In [ ]:
dfc = df_quantile.reset_index().drop(columns=["gender"])
dfc = dfc.groupby(["method", "quantile", "dataset"]).agg({"wer": "mean", "sim": "mean"})
dfc = dfc.reset_index()
dfc

In [ ]:
px.line(
    dfc,
    x="quantile",
    y="wer",
    color="method",
    facet_col="dataset",
    title="WER per quantile",
)

In [ ]:
px.bar(
    dfc.query("quantile == 1"),
    x="dataset",
    color="method",
    barmode="group",
    y="wer",
    # facet_col="dataset",
    title="WER of last quantile",
)

In [ ]:
px.line(
    dfc,
    x="quantile",
    y="sim",
    color="method",
    facet_col="dataset",
    title="Speaker sim per quantile",
)

In [ ]:
px.scatter(
    dfc.loc[dfc["quantile"].isin([1.0])],
    x="wer",
    y="sim",
    color="method",
    facet_col="dataset",
    # title="Speaker sim per quantile",
)

## Sample-level


In [ ]:
dfc = df.loc[df["sample_id"] == "en_speaker_5_text_21"]
dfc

In [ ]:
df

In [ ]:
dfd = df.reset_index()
dfd = dfd.loc[dfd["method"].isin(["16s_context", "opensourced"])]
# dfd = dfd.query("dataset == 'wikibooks_fr'")

In [ ]:
px.histogram(
    dfd.loc[dfd["w_q1.0"] < 1],
    x="w_q1.0",
    color="method",
    barmode="group",
    facet_col="dataset",
)

In [ ]:
px.histogram(
    dfd,
    x="s_q1.0",
    color="method",
    barmode="group",
    facet_col="dataset",
)

In [ ]:
df

In [ ]:
df.sort_values("s_q1.0").head(30)  # .query("method == '16s_context'")